In [ ]:
# 官方文档教程
# https://python.langchain.com/docs/tutorials/llm_chain/

In [27]:
# 使用聊天模型和提示模板构建简单的 LLM 应用程序
# 在本快速入门中，我们将向您展示如何使用 LangChain 构建一个简单的 LLM 应用程序。该应用程序会将文本从英语翻译成另一种语言。这是一个相对简单的 LLM 应用程序——只需一个 LLM 调用加上一些提示即可。尽管如此，这仍然是开始使用 LangChain 的好方法——只需一些提示和一个 LLM 调用就可以构建许多功能！

In [28]:
# 安装 LangChain
# conda install langchain -c conda-forge

In [29]:
# 使用 LangChain 构建的许多应用程序都包含多个步骤，需要多次调用 LLM 函数。随着这些应用程序变得越来越复杂，能够检查链或代理内部究竟发生了什么变得至关重要。最好的方法是使用 LangSmith 。

In [30]:
# import getpass
# import os

# try:
#     # load environment variables from .env file (requires `python-dotenv`)
#     from dotenv import load_dotenv

#     load_dotenv()
# except ImportError:
#     pass

# os.environ["LANGSMITH_TRACING"] = "true"
# if "LANGSMITH_API_KEY" not in os.environ:
#     os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
#         prompt="Enter your LangSmith API key (optional): "
#     )
# if "LANGSMITH_PROJECT" not in os.environ:
#     os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
#         prompt='Enter your LangSmith Project Name (default = "default"): '
#     )
#     if not os.environ.get("LANGSMITH_PROJECT"):
#         os.environ["LANGSMITH_PROJECT"] = "default"

In [31]:
# 使用语言模型
# pip install -qU "langchain[google-genai]"

In [32]:
# 首先让我们直接使用模型。ChatModel 是 LangChain Runnables 的实例，这意味着它们暴露了一个用于交互的标准接口。为了简单地调用模型，我们可以将消息列表传递给 .invoke 方法。

In [33]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [34]:
# 请注意，ChatModel 接收消息对象作为输入，并生成消息对象作为输出。除了文本内容外，消息对象还传达对话角色并保存重要数据，例如工具调用次数和令牌使用次数。

In [35]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--8bae7c8b-6291-4e58-b33d-5ed79ef33808-0', usage_metadata={'input_tokens': 9, 'output_tokens': 3, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [36]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Chinese"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='嗨！ (Hāi!)\n\n你好！ (Nǐ hǎo!) - This is the more formal and common greeting.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--d117f50b-d9ed-4a9c-91db-209960beb52e-0', usage_metadata={'input_tokens': 9, 'output_tokens': 28, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}})

In [37]:
# LangChain 还支持通过字符串或 OpenAI 格式输入聊天模型。以下是等效的：

In [38]:
model.invoke("Hello")



AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--d7755c59-0466-4ad9-91ac-07cc824d207a-0', usage_metadata={'input_tokens': 1, 'output_tokens': 10, 'total_tokens': 11, 'input_token_details': {'cache_read': 0}})

In [39]:
model.invoke([{"role": "user", "content": "Hello"}])



AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--9a0ab176-c3fa-48cf-9117-82e0a0dede9b-0', usage_metadata={'input_tokens': 1, 'output_tokens': 10, 'total_tokens': 11, 'input_token_details': {'cache_read': 0}})

In [40]:
model.invoke([HumanMessage("Hello")])


AIMessage(content='Hello there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--484ddb06-cdf2-4b3d-8c3b-0fd0f411a680-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [41]:
# 流式调用Streaming
# 由于聊天模型是 Runnable 对象 ，它们对外暴露了一个标准接口，其中包含异步和流式调用模式。这使我们能够从聊天模型中流式传输单个令牌：

In [42]:
for token in model.stream(messages):
    print(token.content, end="|")

嗨|！ (Hāi!) 

你好！ (Nǐ hǎo!)| - This is a more formal and common greeting.

Both are perfectly acceptable ways| to say "Hi!" in Chinese. The first is a direct transliteration, the second is the standard greeting.
|

In [43]:
#  提示模板
# 现在，我们将一个消息列表直接传递给语言模型。这个消息列表从何而来？通常，它由用户输入和应用程序逻辑组合而成。应用程序逻辑通常接收原始用户输入，并将其转换为可传递给语言模型的消息列表。常见的转换包括添加系统消息或使用用户输入格式化模板。
# 提示模板是 LangChain 中一个旨在协助这种转换的概念。它们接收原始用户输入并返回可传递给语言模型的数据（提示）。
# 让我们在这里创建一个提示模板。它将接受两个用户变量：
# language ：将文本翻译成的语言
# text ：要翻译的文本
# 请注意， ChatPromptTemplate 在单个模板中支持多种消息角色 。我们将 language 参数格式化为系统消息，将用户 text 格式化为用户消息。

In [44]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [45]:
# 这个提示模板的输入是一个字典。我们可以单独试用一下这个提示模板，看看它能做什么。

In [46]:
prompt = prompt_template.invoke({"language": "Chinese", "text": "Saito Asuka"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Chinese', additional_kwargs={}, response_metadata={}), HumanMessage(content='Saito Asuka', additional_kwargs={}, response_metadata={})])

In [47]:
# 我们可以看到它返回了一个包含两条消息的 ChatPromptValue 。如果我们想直接访问这些消息，我们可以这样做：

In [48]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Chinese', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Saito Asuka', additional_kwargs={}, response_metadata={})]

In [49]:
# 最后，我们可以在格式化的提示上调用聊天模型：

In [50]:
response = model.invoke(prompt)
print(response.content)

斋藤飞鸟 (Zhāi téng Fēiniǎo)


In [51]:
# 结论
# 在本教程中，您学习了如何创建第一个简单的 LLM 应用程序。您学习了如何使用语言模型、如何创建提示模板，以及如何在您使用 LangSmith 创建的应用程序中获得出色的可观察性。